# 2º Trabalho Prático: Planejamento de Caminhos

##### Importação de Bibliotecas

In [1]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import math
import time
import numpy as np
import heapq
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D

#### Funções Compartilhadas

In [2]:
# Normalize angle to the range [-pi,pi)
def normalize_angle (angle):
    return np.mod (angle + np.pi, 2 * np.pi) - np.pi

In [3]:
client = RemoteAPIClient()
sim = client.require('sim')

class CoppeliaObject:
    
    def __init__ (self, name):
        self.name = name
        self.handle = sim.getObject (name)
        self.update_location ()
        
    def new_location (self, position, orientation):
        sim.setObjectPosition (self.handle, position)
        sim.setObjectOrientation (self.handle, orientation)
        self.update_location ()
        
    def update_location (self):
        self.global_position = sim.getObjectPosition (self.handle)
        self.global_orientation = sim.getObjectOrientation (self.handle)

class CoppeliaRobot (CoppeliaObject):
    
    def __init__ (self, name, length, width, wheels):
        # wheel_joints recebe um vetor de objetos do tipo Coppelia_Wheel
        self.length = length
        self.width = width
        self.wheels = wheels
        CoppeliaObject.__init__ (self, name)

    def calc_joint_velocity (self, position):
        delta_pos = np.array (position) - np.array (self.global_position[:2])
        print("Variação de posição: ", delta_pos)
        delta_ori = normalize_angle (np.arctan2 (delta_pos[1], delta_pos[0]) - self.global_orientation[2])
        print("Variação de orientação: ", delta_ori)
        t = (np.linalg.norm (delta_pos) + abs (delta_ori / np.pi)) * 1.5
        print ("Tempo: ", t)
        q_dot = np.array ((delta_pos[0], delta_pos[1], delta_ori)) / t
        return q_dot, t

    def inverse_kinematics_omnidirectional (self, velocity):
        referential_distance = (self.length + self.width) / 2
        matrix = [[-1, -1, -referential_distance],
                  [-1,  1, -referential_distance],
                  [-1, -1,  referential_distance],
                  [-1,  1,  referential_distance]]
        angular_velocity = matrix @ velocity / self.wheels[0].radius
        return angular_velocity
    
    def follow_path (self, path): 
        for stop in path:
            print(stop)
            joint_velocity, t = self.calc_joint_velocity (stop)
            print (joint_velocity)
            wheels_velocity = self.inverse_kinematics_omnidirectional (joint_velocity)
            print (wheels_velocity)
            for i in range (4):
                self.wheels[i].set_velocity (wheels_velocity[i])
            sim.wait(t)
            for i in range (4):
                print (sim.getJointVelocity (self.wheels[i].handle))
            initial_pos = np.array(self.global_position)
                
class CoppeliaWheel (CoppeliaObject):
    
    def __init__ (self, name, radius):
        self.radius = radius
        CoppeliaObject.__init__ (self, name)

    def set_velocity (self, velocity):
        sim.setJointTargetVelocity (self.handle, velocity)

class CoppeliaLaser (CoppeliaObject):
    
    def __init__ (self, name, laser_range_data, laser_angle_data):
        self.laser_range_data = laser_range_data
        self.laser_angle_data = laser_angle_data

#### Funções Roadmap

In [4]:
def read_binary_image (image_address):
    
    image = mpimg.imread (image_address)
    for row in range(len(image)):
        for col in range(len(image[row])):
            image[row][col] = 1 if image[row][col] < 0.3 else 0
            
    return image

def print_world_grid (world_grid):
    
    x_len = len (world_grid[0])
    y_len = len (world_grid)
    
    figure = plt.figure (figsize = (15, 15 * y_len / x_len), dpi = 100)
    ax = figure.add_subplot (111, aspect = 'equal')

    ax.grid (which = 'major', axis = 'both', linestyle = '-', color = 'k', linewidth = .5)
    ax.set_xticks (np.arange (0, x_len))
    ax.set_yticks (np.arange (0, y_len))
    
    ax.imshow (1 - world_grid, cmap = 'Blues', extent = (0, x_len, 0, y_len), alpha = 0.6)
        

In [5]:
class World:
    
    def __init__ (self, image_address, real_dimensions):
        self.image_address = image_address
        self.world_matrix = read_binary_image (self.image_address)
        self.real_dimensions = real_dimensions
        self.scale = self.world_matrix.shape[:2] / self.real_dimensions
        
    def is_cell_area_free (self, row, col, cell_size):
        area_size = cell_size * self.scale
        left = int (col * area_size[0])
        right = int ((col + 1) * area_size[0])
        top = int (row * area_size[1])
        bottom = int ((row + 1) * area_size[1])
        return 1 if np.sum (self.world_matrix[top:bottom, left:right]) == 0 else 0
        
    def discretize_world (self, cell_size = 1):
        num_rows, num_cols = (self.real_dimensions / cell_size).astype(int)
        grid = np.zeros ((num_rows, num_cols))
        for row in range (num_rows):
            for col in range (num_cols):
                grid[row][col] = self.is_cell_area_free (row, col, cell_size)

        print_world_grid(grid)
        return grid

In [6]:
class Node:

    def __init__ (self, grid_position, parent, current_cost):
        self.grid_position = grid_position
        self.parent = parent
        self.current_cost = current_cost
        self.priority_score = current_cost #If I don't call calc_priority_score, the algorithm works like BFS by default

    def __lt__ (self, other):
        return self.priority_score < other.priority_score

    def manhattan_distance (self, goal_pos):
        return abs (self.grid_position[0] - goal_pos[0]) + abs (self.grid_position[1] - goal_pos[1])
        
    def calc_priority_score (self, goal_pos):
        self.heuristic_cost = self.manhattan_distance (goal_pos)
        self.priority_score = self.current_cost + self.heuristic_cost

In [7]:
def is_cell_invalid (cell, world):
    
    if cell [0] < 0: return True
    if cell [1] < 0: return True
    if cell [0] >= len (world[0]): return True
    if cell [1] >= len (world): return True
        
    return False

def reconstruct_path_to (last_node, world):

    path = [last_node.grid_position]
    world[last_node.grid_position[1]][last_node.grid_position[0]] = 3
    while last_node.parent is not None:
        path.append(last_node.parent.grid_position)
        last_node = last_node.parent
        world[last_node.grid_position[1]][last_node.grid_position[0]] = 3

    return path, world

def A_star (start, end, binary_world):

    frontier = []
    path = []
    visited_map = np.zeros ((len(binary_world), len(binary_world[0])))
    binary_world[start[1]][start[0]] = 1
    heapq.heappush (frontier, Node (start, None, 0))

    while len(frontier) > 0:
        current_node = heapq.heappop(frontier)

        if not (current_node.grid_position - end).any():
            path, binary_world = reconstruct_path_to (current_node, binary_world)
            break

        for shift_one in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            neighbor_position = (current_node.grid_position + shift_one).astype(int)
            
            if is_cell_invalid (neighbor_position, binary_world): continue
            if binary_world[neighbor_position[1]][neighbor_position[0]] == 0: continue
            if binary_world[neighbor_position[1]][neighbor_position[0]] == 2: continue
            else: binary_world[neighbor_position[1]][neighbor_position[0]] = 2

            neighbor = Node (neighbor_position, current_node, current_node.current_cost + 1)
            neighbor.calc_priority_score (end)
            heapq.heappush (frontier, neighbor)

    print_world_grid (binary_world)
    return path

In [8]:
#moves the referential from the center of the world to the top left corner of the grid and finds the correspondent cell
def find_grid_position (global_pos, cell_size, grid_size):

    x_pos = np.floor (global_pos[0] / cell_size + grid_size[0] / 2)
    y_pos = np.floor (-global_pos[1] / cell_size + grid_size[1] / 2)

    return np.array ((x_pos, y_pos)).astype(int)

def find_map_position (grid_pos, cell_size, grid_size):

    x_pos = (grid_pos[0] + 0.5 - grid_size[0] / 2) * cell_size
    y_pos = -(grid_pos[1] + 0.5 - grid_size[1] / 2) * cell_size

    return np.array ((x_pos, y_pos))

def find_goal (robot, goal, world_map, cell_size):

    robot_cell = find_grid_position (robot.global_position, cell_size, [len (world_map[0]), len (world_map)])
    goal_cell = find_grid_position (goal.global_position, cell_size, [len (world_map[0]), len (world_map)])
    
    path_to_goal = A_star (robot_cell, goal_cell, world_map)

    path_to_goal = [find_map_position (stop, cell_size, [len (world_map[0]), len (world_map)]) for stop in path_to_goal]
    
    robot.follow_path (np.flip (path_to_goal, 0))

In [ ]:
sim.loadScene ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/cave_youbot.ttt")
sim.startSimulation ()

KUKA_youbot = CoppeliaRobot ("/youBot", 0.58, 0.38, [])

KUKA_youbot.wheels.append (CoppeliaWheel ("/youBot/Joint/rollingJoint_fl", 0.0475))
KUKA_youbot.wheels.append (CoppeliaWheel ("/youBot/rollingJoint_rl", 0.0475))
KUKA_youbot.wheels.append (CoppeliaWheel ("/youBot/rollingJoint_rr", 0.0475))
KUKA_youbot.wheels.append (CoppeliaWheel ("/youBot/Joint/rollingJoint_fr", 0.0475))

goal = CoppeliaObject ("/Goal")

world = World ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/imagens/cave.png", np.array([40,40]))
world_map = world.discretize_world (0.8) # Cell size must be the division of real_dimension by a full number

find_goal (KUKA_youbot, goal, world_map, 0.8)

sim.stopSimulation ()
time.sleep (5)

[ -1.2 -13.2]
Variação de posição:  [-0.37510536 -0.12504207]
Variação de orientação:  0.32176710191445146
Tempo:  0.7467294173519267
[-0.50233103 -0.16745298  0.43090187]
[ 9.74633924  2.69568747 18.45509282 11.40444104]
7.060365568672522
2.6956306347690213
16.816335041458967
11.39372594823794
[ -2.  -13.2]
Variação de posição:  [-1.17510536 -0.12504207]
Variação de orientação:  0.10601020435093744
Tempo:  1.8232253156067384
[-0.6445201  -0.06858289  0.05814432]
[14.42513083 11.53743009 15.60025823 12.71255749]
10.83876047019239
11.537285323958034
15.600084949735868
11.417188597567108
[ -2.8 -13.2]
Variação de posição:  [-1.97510536 -0.12504207]
Variação de orientação:  0.0632245631584265
Tempo:  2.9987768277954268
[-0.658637   -0.04169769  0.02108345]
[14.53083429 12.77514208 14.95694192 13.20124972]
11.430164282182327
12.774934509763321
14.332192071745522
12.041349562009788
[ -3.6 -13.2]
Variação de posição:  [-2.77510536 -0.12504207]
Variação de orientação:  0.04502791755704383
Tem

In [ ]:
a = np.array((0, 3))
b = np.array((0, 4))

#### Funções Campos Potenciais

Neste trecho, foi-se definido as funções de atração e repulsão para locomoção do robô. Com efeito, elas se baseiam nas leituras de posição atual do robô, do goal e do obstáculo (obtida com o Laser). No caso, a força de atração é baseada na diferença entre a posição do 
goal e do robô, multiplicada por um ganho constante. Note que esta função é retorna um vetor. 

Por fim, a foça de repulsão atua com a posição atual do robô, posição do obstáculo, um fator R para distância máxima permitida e um ganho constante. Neste caso,ela é necessária para que se evite a colisão do robô e este consiga efetuar sua trajetória sem ficar preso. No caso, note que esta também retorna um vetor e é inversalmente proporcional com a distância entre o robô e o obstáculo, i.e., quanto mais próxima do obstáculo, maior deve ser.

In [ ]:
# Usar nomes de funções e atributos mais descritivos
def att_force (robotPosition, goal, katt):
    return katt * (goal - robotPosition)

def rep_force(robotPosition, obs, R=5, krep=.1):
    
    # Obstáculo: (x, y, r)
    v = robotPosition[0:2] - obs[0:2]
    d1 = np.linalg.norm(v)
    
    rep = krep * (1/d1**2) * ((1/d1) - (1/R)) * (v/d1)    
    
    return rep

#### Função de leitura de Laser

    Esta é uma das principais funções para o funcionamento deste método. Funciona recebendo como parâmetros as informações necessárias do
Laser, como seu nome e o que ele deve pegar e retorna um vetor com as informações de distância e ângulos captadas pelo Laser. No caso, a sua
limitação se dá pelo fato de que o Laser não captura informações que estejam a mais de 5 metros de distância, por isso a anulação de valores
com o valor 5 na função de repulsão. 

In [ ]:
def read_sensor_data (range_data_signal_id = "hokuyo_range_data", angle_data_signal_id = "hokuyo_angle_data"):
    
    while sim.getStringSignal (range_data_signal_id) == None:
        continue
        
    string_range_data = sim.getStringSignal (range_data_signal_id)
    string_angle_data = sim.getStringSignal (angle_data_signal_id)

    raw_range_data = sim.unpackFloatTable (string_range_data)
    raw_angle_data = sim.unpackFloatTable (string_angle_data)

    return np.array ([raw_angle_data, raw_range_data]).T

#### Funções para o funcionamento do método.
Primeiramente, definiu-se o caminho do robô, dos componentes e do Laser, para que se obtenha o Handle destes objetos. A partir daí, utilizando as funções de getObjectPosition e getObjectOrientation, obtivemos as posições iniciais do robô. A partir daí, fez-se o Path do Goal que é um referenceFrame obtido do Coppeliae obteve sua posição e orientação. Note que este é fixo durante todo código. Após isso, define-se cotas superiores de velocidade que o robô pode alcançar e realiza-se um laço de interação while que continua rodando até que a posição final do robô esteja com erro de 0.05 em relação ao Goal. Dentro desta interação, busca-se atualizar constantemente a posição do robô e sua orientação, para que se possa mudar o valor da força de atração entre o robô e o Goal a medida que este se aproxima do objetivo.
    
Dentro deste while, chama-se continuamente a função de leitura do laser baseada na posição atual do robô que retorna o vetor com valores de distância e ângulos. Para que se tenha um melhor controle sobre estes valores, realiza-se um outro laço de iteração para filtrar os valores que são menores que uma tolerância de 5, são removidos (pois retornariam valores espúrios e perderíamos memória calculando valores não válidos devido a distância). Daí, realizamos o cálculo da força de repulsão baseada nesses valores filtrados.
    
Por fim, realizamos os cálculos somando todas as forças de repulsão e definindo fx e fy como sendo o primeiro e o segundo valor (respectivamente) do vetor da soma da força de atração e todas as de repulsão. Fizemos isso para aplicar o controlador de Lucas-Orio, e a partir dos valores calculados de velocidade linear e ângular, aplicamos as restrições de velocidades máximas e mínimas permitidas, para que se tenha um limite para estas. Após isso, definimos as velocidades da roda do robô diferencial e ele continua andando até que esteja a uma distância menor que 0.05 do Goal
    

In [ ]:
sim.loadScene ("C:/Users/tatia/Desktop/UFMG/RM/TP2/mapas/cave_pioneer.ttt")
sim.startSimulation ()

robotname = './Pioneer_p3dx'
robotname_R_Motor = './Pioneer_p3dx/Pioneer_p3dx_rightMotor'
robotname_L_Motor = './Pioneer_p3dx/Pioneer_p3dx_leftMotor'
laser_range_data = "hokuyo_range_data"
laser_angle_data = "hokuyo_angle_data"

robotHandle = sim.getObjectHandle(robotname)    

robotLeftMotorHandle  = sim.getObjectHandle(robotname_L_Motor)
robotRightMotorHandle = sim.getObjectHandle(robotname_R_Motor)

robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)

goalFrame = sim.getObjectHandle('./Goal')

position_goal = sim.getObjectPosition(goalFrame, sim.handle_world)
orientation_goal = sim.getObjectOrientation(goalFrame, sim.handle_world) 

sim.setObjectPosition(goalFrame, [position_goal[0], position_goal[1], 0], sim.handle_world)
sim.setObjectOrientation(goalFrame, [orientation_goal[0], orientation_goal[1], orientation_goal[2]], sim.handle_world) 

L = 0.381
r = 0.0975
maxv = 1.5
maxw = np.deg2rad(45)

position_goal = sim.getObjectPosition(goalFrame, sim.handle_world)
orientation_goal = sim.getObjectOrientation(goalFrame, sim.handle_world) 

rho = np.inf
     
while rho > .05:

while True :
        
    while rho > .05:


        robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
        robotPos1 = np.array([robotPos[0], robotPos[1], robotPos[2]])
        robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)        
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])        
        laser_data = read_sensor_data(laser_range_data, laser_angle_data)

        força_RepulsTot = [0,0]
        força_AtracTot = att_force(robotPos1, position_goal, 0.03)
        print("Força de atração: ", força_AtracTot)

        for i in laser_data:

            if i[0] < 5:
                f2 = 0
                d1 = i[0]  # Correção aqui
                ang1 = i[1]  # Correção aqui
                x = d1 * np.cos(ang1)
                y = d1 * np.sin(ang1)
                distLaser = np.array([x ,y, 0])
                #print("CCCCCCCCCCCCCCCCCCCCCCC ",distLaser)
                f2 = rep_force(robotConfig, distLaser, 5, 0.05)
                #print("AAAAAAAAAAAAAAAAAAA",f2)
                força_RepulsTot += f2   


        fx,fy = força_AtracTot[:2] - força_RepulsTot[:2]
        dx1, dy1 = position_goal[:2] - robotConfig[:2]

    dx1, dy1 = position_goal[:2] - robotConfig[:2]

        kr = 1
        kt = 2
        print("Força de Repulsão ",força_RepulsTot)

        print("Força de atração ", fx, " Força de repulsão ", fy, "\n")
        v = kr*(fx*np.cos(robotOri[2]) + fy*np.sin(robotOri[2]))
        w = kt*(np.arctan2(fy,fx) - robotConfig[2])
                
        # Limit v,w to +/- max
        
        print("Velocidade do robô ",v," velocidade angular do robot ", w, "\n")
        v = max(min(v,maxv), -maxv)
        w = max(min(w,maxw), -maxw)

        print("Velocidade do robô 2: ",v," velocidade angular do robot 2: ", w, "\n")

        vr = ((2.0*v) + (w*L))/(2.0*r)
        vl = ((2.0*v) - (w*L))/(2.0*r)
        
        sim.setJointTargetVelocity(robotRightMotorHandle, vr)
        sim.setJointTargetVelocity(robotLeftMotorHandle, vl)

    print("Força de atração ", dx, " Força de repulsão ", dy, "\n")
    v = kr*(dy*np.cos(robotConfig[2]) + dx*np.sin(robotConfig[2]))
    w = kt*(np.arctan2(dy,dx) - robotConfig[2])
            
    # Limit v,w to +/- max
    
    print("Velocidade do robô ",v," velocidade angular do robot ", w, "\n")
    if v > 0 and v > maxv:
        v = maxv
    elif v < 0 and v < -maxv:
        v = -maxv 
    if w > 0 and w > maxw:
        w = maxw
    elif w < 0 and w < -maxw:
        w = -maxw       
    print("Velocidade do robô 2: ",v," velocidade angular do robot 2: ", w, "\n")

    vr = ((2.0*v) + (w*L))/(2.0*r)
    vl = ((2.0*v) - (w*L))/(2.0*r)
    
    sim.setJointTargetVelocity(robotRightMotorHandle, vr)
    sim.setJointTargetVelocity(robotLeftMotorHandle, vl)

sim.setJointTargetVelocity(robotRightMotorHandle, 0)
sim.setJointTargetVelocity(robotLeftMotorHandle, 0)

time.sleep (5)